### 1. Import packages and dataset

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import ast

#load data
file_path = "/Users/butttok/Downloads/books_1.Best_Books_Ever.csv"
df = pd.read_csv(file_path)

print(df.shape)
df.head(3)

(52478, 25)


,bookId,title,series,author,rating,description,language,isbn,genres,characters,...,firstPublishDate,awards,numRatings,ratingsByStars,likedPercent,setting,coverImg,bbeScore,bbeVotes,price
0,2767052-the-hunger-games,The Hunger Games,The Hunger Games #1,Suzanne Collins,4.33,WINNING MEANS FAME AND FORTUNE.LOSING MEANS CE...,English,9780439023481,"['Young Adult', 'Fiction', 'Dystopia', 'Fantas...","['Katniss Everdeen', 'Peeta Mellark', 'Cato (H...",...,NaN,['Locus Award Nominee for Best Young Adult Boo...,6376780,"['3444695', '1921313', '745221', '171994', '93...",96.0,"['District 12, Panem', 'Capitol, Panem', 'Pane...",https://i.gr-assets.com/images/S/compressed.ph...,2993816,30516,5.09
1,2.Harry_Potter_and_the_Order_of_the_Phoenix,Harry Potter and the Order of the Phoenix,Harry Potter #5,"J.K. Rowling, Mary GrandPré (Illustrator)",4.50,There is a door at the end of a silent corrido...,English,9780439358071,"['Fantasy', 'Young Adult', 'Fiction', 'Magic',...","['Sirius Black', 'Draco Malfoy', 'Ron Weasley'...",...,06/21/03,['Bram Stoker Award for Works for Young Reader...,2507623,"['1593642', '637516', '222366', '39573', '14526']",98.0,['Hogwarts School of Witchcraft and Wizardry (...,https://i.gr-assets.com/images/S/compressed.ph...,2632233,26923,7.38
2,2657.To_Kill_a_Mockingbird,To Kill a Mockingbird,To Kill a Mockingbird,Harper Lee,4.28,The unforgettable novel of a childhood in a sl...,English,9999999999999,"['Classics', 'Fiction', 'Historical Fiction', ...","['Scout Finch', 'Atticus Finch', 'Jem Finch', ...",...,07/11/60,"['Pulitzer Prize for Fiction (1961)', 'Audie A...",4501075,"['2363896', '1333153', '573280', '149952', '80...",95.0,"['Maycomb, Alabama (United States)']",https://i.gr-assets.com/images/S/compressed.ph...,2269402,23328,NaN


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52478 entries, 0 to 52477
Data columns (total 25 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   bookId            52478 non-null  object 
 1   title             52478 non-null  object 
 2   series            23470 non-null  object 
 3   author            52478 non-null  object 
 4   rating            52478 non-null  float64
 5   description       51140 non-null  object 
 6   language          48672 non-null  object 
 7   isbn              52478 non-null  object 
 8   genres            52478 non-null  object 
 9   characters        52478 non-null  object 
 10  bookFormat        51005 non-null  object 
 11  edition           4955 non-null   object 
 12  pages             50131 non-null  object 
 13  publisher         48782 non-null  object 
 14  publishDate       51598 non-null  object 
 15  firstPublishDate  31152 non-null  object 
 16  awards            52478 non-null  object

### 2. Preprocess dataset

In [3]:
# convert pages, price to numeric
df['pages'] = pd.to_numeric(df['pages'], errors='coerce')
df['price'] = pd.to_numeric(df['price'].astype(str).str.replace('$', '', regex=False), errors='coerce')

In [4]:
language_counts = df['language'].value_counts()

print(language_counts)

print("\n" + "="*50 + "\n")

# calculate percentage of each language
language_percentage = df['language'].value_counts(normalize=True) * 100

print("Percentage of each language (%)")

language_summary = pd.DataFrame({
    'Count': language_counts,
    'Percentage (%)': language_percentage.round(2)
})

print(language_summary)

language
English                                  42661
Arabic                                    1038
Spanish                                    687
French                                     579
German                                     528
                                         ...  
Aromanian; Arumanian; Macedo-Romanian        1
Basque                                       1
Faroese                                      1
Iranian (Other)                              1
Norwegian Nynorsk; Nynorsk, Norwegian        1
Name: count, Length: 81, dtype: int64


Percentage of each language (%)
                                       Count  Percentage (%)
language                                                    
English                                42661           87.65
Arabic                                  1038            2.13
Spanish                                  687            1.41
French                                   579            1.19
German                              

In [5]:
#create id column
df["id"] = range(1, len(df) + 1)

#drop coverImg column
df = df.drop('coverImg',axis=1)

#drop language not English
df = df[df['language'] == 'English'].copy() 

# create a column for average bbeScore
df['bbeScore'] = pd.to_numeric(df['bbeScore'], errors='coerce')
df['bbeVotes'] = pd.to_numeric(df['bbeVotes'], errors='coerce')
df['avg_bbeScore'] = np.where(
    (df['bbeVotes'].notna()) & (df['bbeVotes'] != 0),
    df['bbeScore'] / df['bbeVotes'],
    np.nan
)

# drop rows with missing value avg_bbeScore
df['avg_bbeScore'] = pd.to_numeric(df['avg_bbeScore'], errors='coerce')
df = df.dropna(subset=['avg_bbeScore']).copy()

# create publish year column
df['publishDate_parsed'] = pd.to_datetime(df['publishDate'], errors='coerce')
df['year'] = df['publishDate_parsed'].dt.year
df.loc[df['year'] < 1925, 'year'] = np.nan
df.loc[df['year'] > 2025, 'year'] = np.nan
df = df.drop('publishDate_parsed', axis=1)

<ipython-input-5-5691eaeee9f0>:24: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['publishDate_parsed'] = pd.to_datetime(df['publishDate'], errors='coerce')


### 3. Feature Engineering

In [6]:
#convert award to binary feature
df['has_award'] = df['awards'].apply(lambda x: 0 if x == '[]' or pd.isna(x) else 1)

#drop missing description, year
df = df.dropna(subset=['description']).copy()
df = df.dropna(subset=['year']).copy()

#divide ratingsByStars into separate columns
def parse_ratings_list(val):
    try:
        if pd.isna(val) or val in ('[]', '0'):
            return [None] * 5
        parsed_list = ast.literal_eval(val)
        if len(parsed_list) == 5:
            return [int(x) if x is not None and str(x).isdigit() else None for x in parsed_list]
        else:
            return [None] * 5
    except (ValueError, SyntaxError, TypeError):
        return [None] * 5

df['ratings_list'] = df['ratingsByStars'].apply(parse_ratings_list)
ratings_columns = ['rating_5star', 'rating_4star', 'rating_3star', 'rating_2star', 'rating_1star']
df[ratings_columns] = df['ratings_list'].apply(pd.Series)
for col in ratings_columns:
    df[col] = pd.to_numeric(df[col], errors='coerce').astype('Int64', errors='ignore')

print(df[['ratingsByStars'] + ratings_columns].head(10).to_markdown(index=False, numalign="left", stralign="left"))

#remove outliers
df.dropna(subset=['likedPercent', 'price'], inplace=True)
def remove_outliers(df, col):
    q1 = df[col].quantile(0.25)
    q3 = df[col].quantile(0.75)
    iqr = q3 - q1
    lower = q1 - 1.5 * iqr
    upper = q3 + 1.5 * iqr
    return df[(df[col] >= lower) & (df[col] <= upper)]

for col in ['likedPercent', 'price']:
    before = df.shape[0]
    df = remove_outliers(df, col)
    after = df.shape[0]
    print(f"{col}: Removed {before - after} outliers")

| ratingsByStars                                        | rating_5star   | rating_4star   | rating_3star   | rating_2star   | rating_1star   |
|:------------------------------------------------------|:---------------|:---------------|:---------------|:---------------|:---------------|
| ['3444695', '1921313', '745221', '171994', '93557']   | 3444695        | 1921313        | 745221         | 171994         | 93557          |
| ['1593642', '637516', '222366', '39573', '14526']     | 1593642        | 637516         | 222366         | 39573          | 14526          |
| ['2363896', '1333153', '573280', '149952', '80794']   | 2363896        | 1333153        | 573280         | 149952         | 80794          |
| ['1617567', '816659', '373311', '113934', '76770']    | 1617567        | 816659         | 373311         | 113934         | 76770          |
| ['1751460', '1113682', '1008686', '542017', '548674'] | 1751460        | 1113682        | 1008686        | 542017         | 548674         |

In [7]:
#check missing values
print("\nMissing Values：")
print(df.isna().sum().sort_values(ascending=False).head(10))


Missing Values：
edition             26381
series              14888
firstPublishDate     9258
pages                 223
publisher             152
rating_1star           82
rating_2star           82
rating_3star           82
rating_4star           82
rating_5star           82
dtype: int64


In [8]:
print(df.shape)
df.head(3).T

(28644, 34)


,0,1,5
bookId,2767052-the-hunger-games,2.Harry_Potter_and_the_Order_of_the_Phoenix,19063.The_Book_Thief
title,The Hunger Games,Harry Potter and the Order of the Phoenix,The Book Thief
series,The Hunger Games #1,Harry Potter #5,NaN
author,Suzanne Collins,"J.K. Rowling, Mary GrandPré (Illustrator)",Markus Zusak (Goodreads Author)
rating,4.33,4.5,4.37
description,WINNING MEANS FAME AND FORTUNE.LOSING MEANS CE...,There is a door at the end of a silent corrido...,Librarian's note: An alternate cover edition c...
language,English,English,English
isbn,9780439023481,9780439358071,9780375831003
genres,"['Young Adult', 'Fiction', 'Dystopia', 'Fantas...","['Fantasy', 'Young Adult', 'Fiction', 'Magic',...","['Historical Fiction', 'Fiction', 'Young Adult..."
characters,"['Katniss Everdeen', 'Peeta Mellark', 'Cato (H...","['Sirius Black', 'Draco Malfoy', 'Ron Weasley'...","['Liesel Meminger', 'Hans Hubermann', 'Rudy St..."


### 5. Export dataset

In [9]:
df.to_csv("cleaned_books_dataset.csv", index=False)
print("\nSuccessfully export cleaned_books_dataset.csv")


Successfully export cleaned_books_dataset.csv
